In [ ]:
import requests
from dataclasses import dataclass
import pandas as pd
import panel as pn
from tabulate import tabulate
from markdown import markdown
# !pip install openpyxl
# !pip install tabulate

pn.extension()

In [ ]:
DOC_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vStz17Gi-O3tJjWcT_F0zYj4eCVuiiaU9ewpKTLlu_qRak-Cd0NHG3oQa0lcVFmWC2TFK3ecZHvdPxT/pub?output=xlsx"

In [ ]:
@dataclass
class ProductCard:
    name: str
    data_overview: dict
    var_table: pd.DataFrame

    @property
    def variables(self):
        return list(self.var_table.index)
    
    def variable_description_markdown(self, variable, **kwargs):
        desc = self.var_table.loc[variable]["Description"]
        return pn.pane.Markdown(desc, **kwargs)
    
    def variables_table_html(self):
        df = self.var_table.copy()
#         if "Description" in df.columns:
#             df["Description"] = df["Description"].apply(lambda x: markdown(x))
        table = tabulate(
            df.values, df.columns,
            tablefmt="html",
        )
#         table = table.replace("\n", "<br>").replace("right", "left")
        return table
#         return pn.pane.HTML(self.var_table.to_html(na_rep="-"), width_policy="max")
    
    @staticmethod
    def _df_to_markdown(df):
        return tabulate(df.values,df.columns, tablefmt="pipe")
    
    def variables_table_markdown(self, **kwargs):
        if isinstance(self.var_table, pd.DataFrame):
            return pn.pane.Markdown(
                self._df_to_markdown(self.var_table),
                style={
                    "border": "1px solid black",
                },
                **kwargs
            )
        else:
            return pn.pane.Markdown("", **kwargs)
    
    def panel_dataframe_widget(self):
        return pn.widgets.DataFrame(self.var_table.drop(columns=["Variable Name"]))
    
    def panel_dataframe_pane(self):
        return pn.pane.DataFrame(self.var_table.drop(columns=["Variable Name"]))
    
    def product_info_short(self, **kwargs):
        name = f"## {self.name}"
        info = self.data_overview
        content = [
            name,
            info["Description (short)"],
            f'<{info["Link: HTTP"]}>'
        ]
        return pn.pane.Markdown(
            "\n\n".join(content), **kwargs
        )
    
    def product_info_long(self, **kwargs):
        content = [self.product_info_short().object]
        var_table = self.variables_table_html()
        content.extend([
            f"<details>\n\n{var_table}\n\n</details>"
        ])
        return pn.pane.Markdown(
            "\n\n".join(content), **kwargs
        )

    def make_card_pane(self, include_var_table=False):
        widget_button = pn.widgets.Button(name=f"View details: {self.name}", button_type="primary")
        self.button_to_trigger_details = widget_button
        pane_card = pn.Card(
            widget_button,
            self.product_info_long() if include_var_table else self.product_info_short(),
            collapsed=True,
            title=self.name,# + ": " + overview.loc[name]["Description (short)"]
            sizing_mode="stretch_width"
        )
        self.pane_card = pane_card
        self.button_on_click = widget_button.on_click
        return pane_card
    

@dataclass
class CardCatalog:
    overview: pd.DataFrame
    cards: dict
        
    @property
    def names(self):
        return list(self.overview.index.dropna())
        
    def product_info_short(self, name, **kwargs):
        return self.cards[name].product_info_short(**kwargs)
    
    def product_info_long(self, name, **kwargs):
        return self.cards[name].product_info_long(**kwargs)
    
    def cards_filtered(self, name=""):
        return {k: v for (k,v) in self.cards.items() if name in k}
    
    def build_card_panes(self):
        for card in self.cards.values():
            card.make_card_pane()

In [ ]:
def load_data(url=DOC_URL):
    xl_doc = requests.get(url).content
    overview = pd.read_excel(xl_doc, "Overview")#, engine="odf")
    overview = overview.set_index("Name").fillna("-")
    names = list(overview.index.dropna())
    details = {}
    missing_sheets = []
    for name in names:
        try:
            details[name] = pd.read_excel(xl_doc, name).set_index("Variable Name", drop=False).fillna("-")
        except Exception:
            missing_sheets.append(name)
            details[name] = pd.DataFrame()
    return overview, details

# OVERVIEW, DETAILS = load_data()

In [ ]:
def build_catalog(url=DOC_URL, reload=False):
    if ("OVERVIEW" not in globals()) or ("DETAILS" not in globals()):
        reload = True
    if reload:
        overview, details = load_data()
    else:
        overview, details = OVERVIEW.copy(), DETAILS.copy()
    catalog = CardCatalog(overview, dict())
    for name in catalog.names:
#         try:
        catalog.cards[name] = ProductCard(
            name,
            dict(overview.loc[name]),
            details.get(name)
        )
#         except Exception:
#             pass
    catalog.build_card_panes()
    return catalog
    
if "CATALOG" not in pn.state.cache:
    CATALOG = catalog = pn.state.cache["CATALOG"] = build_catalog(reload=False)
else:
    CATALOG = catalog = pn.state.cache["CATALOG"]

In [ ]:
# from IPython.display import HTML
# catalog.cards["MAGx_LR_1B"].variables_table_html()
# HTML(catalog.cards["MAGx_LR_1B"].variables_table_html())

In [ ]:
catalog.product_info_long("MAGx_LR_1B", width=500, sizing_mode="stretch_both")

In [ ]:
reports = [
    catalog.product_info_long(name, width=500, sizing_mode="stretch_both")
    for name in catalog.names
]
reports = [r.object for r in reports]
reports = "\n\n<hr>".join(reports)
report = pn.pane.Markdown(
    reports
)
with open("output/report.html", "w") as f:
    f.write(markdown(report.object))
# report

In [ ]:
catalog.cards["MAGx_LR_1B"].make_card_pane()

In [ ]:
class Dashboard:
    
    def __init__(self, options=None):
        self.selector_widget = pn.widgets.RadioBoxGroup(
            options=options
        )
        # Create set of possible columns to display
        self.card_stacks = {}
        self.generate_card_stacks(options)
        # Initialise column for cards
        self.card_column = pn.Column(width=400)
        self.card_column.objects = self.card_stacks["All products"]
        # Link filtering of cards
        def change_cards(target, event):
            name_filter = event.new
            target.objects = self.card_stacks[name_filter]
        self.selector_widget.link(self.card_column, callbacks={"value": change_cards})
#         # Some way to do it with js?
#         js_code = """
#             target.objects = source.value
#         """
#         self.selector_widget.jslink
        # Create set of possible detail panes to display
        self.details_html = {}
        self.generate_details_html(CATALOG.names)
        # Initialise html pane for details
        self.detail_pane = pn.pane.HTML(width=500)
        # Link every button, one within each card
        for name in CATALOG.names:
            CATALOG.cards[name].button_to_trigger_details.on_click(
                self.update_details_pane(name)
            )
        self.detail_pane.objects = self.card_detail_html("MAGx_LR_1B")
    
    def display(self):
        return pn.Row(
            pn.Column("Apply filter:", self.selector_widget, width=150),
            self.card_column,
            self.detail_pane,
            height=700,
        )
    
    @staticmethod
    def card_subselection(name_filter):
        _name_filter = "" if name_filter == "All products" else name_filter
        return [card.pane_card for card in CATALOG.cards_filtered(_name_filter).values()]
    
    def generate_card_stacks(self, name_filters, **kwargs):
        for name_filter in name_filters:
            self.card_stacks[name_filter] = self.card_subselection(name_filter)
    
    @staticmethod
    def card_detail_html(name):
        return CATALOG.cards[name].variables_table_html()
    
    def generate_details_html(self, names):
        for name in names:
            self.details_html[name] = self.card_detail_html(name)
    
    def update_details_pane(self, name):
        def _update_details_pane(event):
            self.detail_pane.object = self.card_detail_html(name)
        return _update_details_pane


d = Dashboard(["All products", "MAG", "SHA"])
d.display().servable(title="Swarm card catalogue")

In [ ]:
d.display().save("output/cards.html")